In [3]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [5]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/15 21:28:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)



In [11]:
# 2. Create a temporary view of the DataFrame.
home_sales_df.createOrReplaceTempView("tablex")
print(home_sales_df)


DataFrame[id: string, date: date, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]


In [12]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
q3 = spark.sql("SELECT date, ROUND(AVG(price), 2) AS average_price FROM tablex WHERE bedrooms = 4 GROUP BY date ORDER BY date")
q3.show()


+----------+-------------+
|      date|average_price|
+----------+-------------+
|2019-01-01|     365088.0|
|2019-01-02|    271644.29|
|2019-01-03|    381218.86|
|2019-01-04|    359586.14|
|2019-01-05|    323683.57|
|2019-01-06|    301091.13|
|2019-01-07|     327474.5|
|2019-01-08|    304567.83|
|2019-01-09|     243619.0|
|2019-01-10|     287844.4|
|2019-01-11|    277574.29|
|2019-01-12|     218507.0|
|2019-01-13|    294603.86|
|2019-01-14|    284434.57|
|2019-01-15|     254288.5|
|2019-01-16|     335106.5|
|2019-01-17|    333439.09|
|2019-01-18|    297542.56|
|2019-01-19|    294484.22|
|2019-01-20|    312395.33|
+----------+-------------+
only showing top 20 rows



In [13]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
q4 = spark.sql("SELECT date_built, ROUND(AVG(price), 2) AS average_price FROM tablex WHERE bedrooms = 3 AND bathrooms = 3 GROUP BY date_built ORDER BY date_built")
q4.show()


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [14]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
q5 = spark.sql("SELECT date_built, ROUND(AVG(price), 2) AS average_price FROM tablex WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000 GROUP BY date_built ORDER BY date_built")
q5.show()


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [15]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

q6 = spark.sql("SELECT ROUND(AVG(price), 2) AS average_price FROM tablex WHERE price >= 350000")
q6.show()


print("--- %s seconds ---" % (time.time() - start_time))

+-------------+
|average_price|
+-------------+
|    473796.26|
+-------------+

--- 0.20845389366149902 seconds ---


In [16]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("tablex")


In [18]:
# 8. Check if the table is cached.
spark.catalog.isCached('tablex')

True

In [19]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

q6 = spark.sql("SELECT ROUND(AVG(price), 2) AS average_price FROM tablex WHERE price >= 350000")
q6.show()

print("--- %s seconds ---" % (time.time() - start_time))


+-------------+
|average_price|
+-------------+
|    473796.26|
+-------------+

--- 0.4120500087738037 seconds ---


In [23]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
output_path = "partitioned_data.parquet"
home_sales_df.write.partitionBy("date_built").parquet(output_path)

In [24]:
# 11. Read the formatted parquet data.
parquet_data = spark.read.parquet(output_path)
parquet_data

DataFrame[id: string, date: date, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int, date_built: int]

In [25]:
# 12. Create a temporary table for the parquet data.
parquet_data.createOrReplaceTempView("parquet_table")


In [26]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()

result_cached = spark.sql("SELECT ROUND(AVG(price), 2) AS average_price FROM tablex WHERE view >= 350000")
result_cached.show()

cached_runtime = time.time() - start_time

print("--- %s seconds ---" % (time.time() - start_time))

+-------------+
|average_price|
+-------------+
|         NULL|
+-------------+

--- 0.1845109462738037 seconds ---


In [27]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("tablex")


In [28]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("tablex")



False